## Data Preparation and Dataset Creation

In this notebook, the goal is to combine the given seperate datasets into a single ready-to-use dataset. 
The seperate datasets given within the scope of the competition are as follows:

- Base Station basic information (BSinfo.csv): include configuration parameters and hardware attributes. In more detail, the following information is reported.

- Cell-level data (CLdata.csv): include hour-level counters, including service compliance counters (e.g., load) and energy-saving methods counters (e.g., duration of energy saving mode activation). In more detail, the following information is reported.

- Energy consumption data (ECdata.csv): include hour-level energy consumption specifications (e.g., total energy consumption of the base stations). In more detail, the following information is reported.



In the dataset preparation process, the following steps are followed:

- For Training Dataset:
    - 1) Convert categorical columns to meaningful numerical columns and time columns to datetime.
    - 2) Merge BSinfo and Cell-level on cell-level (so that each multiple cell-level data has the corresponding info attributes)
    - 3) Merge the resulting df and ECdata on ECdata since we need features and the labeled energy outputs for training
    - 4) Separate the resulting dataframe for each cell (at a given time, a given BS can have multiple cells working) so we have 4 different cell-level combined,
     and labeled data
     - 5) Merge all 4 single cell-level dataframes so that we have simultaneously working cells side by side 

#
- For Prediction Dataset: The steps are same as the training dataset, however, the only difference is we don't have the energy outputs to train on. We follow the same
steps to generate a dataset to use model on to and make sumbission predictions.

Also, we use some time-series feature engineering such as:
- 3 different lag values of KPIs (for loads and ESModes)
- 3 different lag values of Energy value (-1h, -2h, -1D)
- Moving average of the Energy (up to previous data point) 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from feature_engine.timeseries.forecasting import LagFeatures

pd.set_option('display.max_columns', 500)

from warnings import filterwarnings
filterwarnings("ignore")

Read the given datasets

In [2]:
df_info = pd.read_csv("data/base_station_basic_info.csv")
df_cell_level = pd.read_csv("data/cell_level.csv")
df_energy_cons = pd.read_csv("data/energy_consumption.csv")
df_prediction = pd.read_csv("data/power_consumption_prediction.csv")

In [3]:
df_info["Mode"] = df_info["Mode"].map({"Mode1":0,"Mode2":1})
df_info = df_info.join(pd.get_dummies(df_info["RUType"],drop_first=True,dtype=np.int32)).drop("RUType",axis=1)
df_info.head()

,BS,CellName,Mode,Frequency,Bandwidth,Antennas,TXpower,Type10,Type11,Type12,Type2,Type3,Type4,Type5,Type6,Type7,Type8,Type9
0,B_0,Cell0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0
1,B_1,Cell0,1,532.0,20,4,6.875934,0,0,0,1,0,0,0,0,0,0,0
2,B_2,Cell0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0
3,B_3,Cell0,1,532.0,20,4,6.875934,0,0,0,1,0,0,0,0,0,0,0
4,B_4,Cell0,1,532.0,20,4,6.875934,0,0,0,1,0,0,0,0,0,0,0


Merge the given datasets to have training and prediction datasets

In [4]:
df_cell_level["Time"] = pd.to_datetime(df_cell_level["Time"])
df_energy_cons["Time"] = pd.to_datetime(df_energy_cons["Time"])
df_prediction["Time"] = pd.to_datetime(df_prediction["Time"])

merge1 = pd.merge(df_cell_level, df_info, how="left",  on=["BS","CellName"])

# df is for training dataset, merge1 is merged on ECdata data to match the given labeled energy datapoints
df = pd.merge(merge1, df_energy_cons, how="right", on=["BS","Time"])

# df_pred is later used when making the predictions (submission), it has the same structure with df, only difference is there are no
# given energy labels.
df_pred = pd.merge(merge1, df_prediction, how="right", on=["BS","Time"])

Separate each cell-level dataframe and merge them side by side to have multiple working cells stacked horizontally 

In [5]:
df_c0 = df[df["CellName"]=="Cell0"].copy()
df_c1 = df[df["CellName"]=="Cell1"].copy()
df_c2 = df[df["CellName"]=="Cell2"].copy()
df_c3 = df[df["CellName"]=="Cell3"].copy()

df_bs = pd.merge(
    pd.merge(
        pd.merge(
            df_c0,df_c1,on=["BS","Time"],how="left",suffixes=("_0","_1")),
        df_c2  ,on=["BS","Time"],how="left"),
df_c2,on=["BS","Time"],how="left",suffixes=("_2","_3"))

df_bs.head()

,Time,BS,CellName_0,load_0,ESMode1_0,ESMode2_0,ESMode3_0,ESMode4_0,ESMode5_0,ESMode6_0,Mode_0,Frequency_0,Bandwidth_0,Antennas_0,TXpower_0,Type10_0,Type11_0,Type12_0,Type2_0,Type3_0,Type4_0,Type5_0,Type6_0,Type7_0,Type8_0,Type9_0,Energy_0,CellName_1,load_1,ESMode1_1,ESMode2_1,ESMode3_1,ESMode4_1,ESMode5_1,ESMode6_1,Mode_1,Frequency_1,Bandwidth_1,Antennas_1,TXpower_1,Type10_1,Type11_1,Type12_1,Type2_1,Type3_1,Type4_1,Type5_1,Type6_1,Type7_1,Type8_1,Type9_1,Energy_1,CellName_2,load_2,ESMode1_2,ESMode2_2,ESMode3_2,ESMode4_2,ESMode5_2,ESMode6_2,Mode_2,Frequency_2,Bandwidth_2,Antennas_2,TXpower_2,Type10_2,Type11_2,Type12_2,Type2_2,Type3_2,Type4_2,Type5_2,Type6_2,Type7_2,Type8_2,Type9_2,Energy_2,CellName_3,load_3,ESMode1_3,ESMode2_3,ESMode3_3,ESMode4_3,ESMode5_3,ESMode6_3,Mode_3,Frequency_3,Bandwidth_3,Antennas_3,TXpower_3,Type10_3,Type11_3,Type12_3,Type2_3,Type3_3,Type4_3,Type5_3,Type6_3,Type7_3,Type8_3,Type9_3,Energy_3
0,2023-01-01 01:00:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,64.275037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01 02:00:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,55.904335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-01 03:00:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,57.698057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-01 04:00:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,55.156951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-01 05:00:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,56.053812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# We can count NaN columns to find out number of simultaneously working cells. 
cwc = len([c for c in df_bs.columns if "_0" in c])
df_bs["Num Working Cell"] = 4-df_bs.isna().sum(1)//cwc 

In [7]:
# Extract some temporal data as feature engineering
df_bs["Hour"] = df_bs["Time"].dt.hour
df_bs["Day"] = df_bs["Time"].dt.day_of_week
df_bs["Hour Sin"] = (np.sin(df_bs["Hour"]*2*np.pi/24))
df_bs["Hour Cos"] = (np.cos(df_bs["Hour"]*2*np.pi/24))
df_bs["Day Sin"] = (np.sin(df_bs["Day"]*2*np.pi/7))
df_bs["Day Cos"] = (np.cos(df_bs["Day"]*2*np.pi/7))
df_bs.drop(columns=["Hour","Day"],inplace=True)
df_bs.drop(columns=["CellName_0","CellName_1","CellName_2","CellName_3"],inplace=True)

df_bs["Antennas"] = df_bs["Antennas_0"]
df_bs = df_bs.drop(columns=["Antennas_0","Antennas_1", "Antennas_2", "Antennas_3"])

df_bs["Energy"] = df_bs["Energy_0"]
df_bs = df_bs.drop(columns=["Energy_0","Energy_1", "Energy_2", "Energy_3"])

In [8]:
# Add the moving average of energy to the features 
df_bs["Avg"] = 0
for BS in df_bs["BS"].unique():
    filt = df_bs[df_bs["BS"]==BS].sort_values("Time")
    cum_sum = 0.
    n = 0
    for idx,row in filt.iterrows():
        df_bs.loc[idx,"Avg"] = cum_sum/max(1,n)
        cum_sum += df_bs.loc[idx,"Energy"]
        n += 1

Finally, we have the training dataset. These datapoints are the labeled (in the sense that corresponding energy consumption is given), and each row shows
which cells are working with the given counters and configurations. For each row, the NaN columns correspond to non-working cell at that time. 

In [9]:
df_bs.head()

,Time,BS,load_0,ESMode1_0,ESMode2_0,ESMode3_0,ESMode4_0,ESMode5_0,ESMode6_0,Mode_0,Frequency_0,Bandwidth_0,TXpower_0,Type10_0,Type11_0,Type12_0,Type2_0,Type3_0,Type4_0,Type5_0,Type6_0,Type7_0,Type8_0,Type9_0,load_1,ESMode1_1,ESMode2_1,ESMode3_1,ESMode4_1,ESMode5_1,ESMode6_1,Mode_1,Frequency_1,Bandwidth_1,TXpower_1,Type10_1,Type11_1,Type12_1,Type2_1,Type3_1,Type4_1,Type5_1,Type6_1,Type7_1,Type8_1,Type9_1,load_2,ESMode1_2,ESMode2_2,ESMode3_2,ESMode4_2,ESMode5_2,ESMode6_2,Mode_2,Frequency_2,Bandwidth_2,TXpower_2,Type10_2,Type11_2,Type12_2,Type2_2,Type3_2,Type4_2,Type5_2,Type6_2,Type7_2,Type8_2,Type9_2,load_3,ESMode1_3,ESMode2_3,ESMode3_3,ESMode4_3,ESMode5_3,ESMode6_3,Mode_3,Frequency_3,Bandwidth_3,TXpower_3,Type10_3,Type11_3,Type12_3,Type2_3,Type3_3,Type4_3,Type5_3,Type6_3,Type7_3,Type8_3,Type9_3,Num Working Cell,Hour Sin,Hour Cos,Day Sin,Day Cos,Antennas,Energy,Avg
0,2023-01-01 01:00:00,B_0,0.487936,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.258819,0.965926,-0.781831,0.62349,4,64.275037,0.000000
1,2023-01-01 02:00:00,B_0,0.344468,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.500000,0.866025,-0.781831,0.62349,4,55.904335,64.275037
2,2023-01-01 03:00:00,B_0,0.193766,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.707107,0.707107,-0.781831,0.62349,4,57.698057,60.089686
3,2023-01-01 04:00:00,B_0,0.222383,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.866025,0.500000,-0.781831,0.62349,4,55.156951,59.292476
4,2023-01-01 05:00:00,B_0,0.175436,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.965926,0.258819,-0.781831,0.62349,4,56.053812,58.258595


## Prediction Dataset

Apply same operations to the prediction data. For making the predictions, feature values are also contained in the CLdata and BSinfo, so we used merged
dataset 

In [10]:
df_pred.drop(columns=["Energy","w"],inplace=True)

df_c0 = df_pred[df_pred["CellName"]=="Cell0"].copy()
df_c1 = df_pred[df_pred["CellName"]=="Cell1"].copy()
df_c2 = df_pred[df_pred["CellName"]=="Cell2"].copy()
df_c3 = df_pred[df_pred["CellName"]=="Cell3"].copy()

df_pred_bs = pd.merge(
    pd.merge(
        pd.merge(
            df_c0,df_c1,on=["BS","Time"],how="left",suffixes=("_0","_1")),
        df_c2  ,on=["BS","Time"],how="left"),
df_c2,on=["BS","Time"],how="left",suffixes=("_2","_3"))

df_pred_bs.head()

,Time,BS,CellName_0,load_0,ESMode1_0,ESMode2_0,ESMode3_0,ESMode4_0,ESMode5_0,ESMode6_0,Mode_0,Frequency_0,Bandwidth_0,Antennas_0,TXpower_0,Type10_0,Type11_0,Type12_0,Type2_0,Type3_0,Type4_0,Type5_0,Type6_0,Type7_0,Type8_0,Type9_0,CellName_1,load_1,ESMode1_1,ESMode2_1,ESMode3_1,ESMode4_1,ESMode5_1,ESMode6_1,Mode_1,Frequency_1,Bandwidth_1,Antennas_1,TXpower_1,Type10_1,Type11_1,Type12_1,Type2_1,Type3_1,Type4_1,Type5_1,Type6_1,Type7_1,Type8_1,Type9_1,CellName_2,load_2,ESMode1_2,ESMode2_2,ESMode3_2,ESMode4_2,ESMode5_2,ESMode6_2,Mode_2,Frequency_2,Bandwidth_2,Antennas_2,TXpower_2,Type10_2,Type11_2,Type12_2,Type2_2,Type3_2,Type4_2,Type5_2,Type6_2,Type7_2,Type8_2,Type9_2,CellName_3,load_3,ESMode1_3,ESMode2_3,ESMode3_3,ESMode4_3,ESMode5_3,ESMode6_3,Mode_3,Frequency_3,Bandwidth_3,Antennas_3,TXpower_3,Type10_3,Type11_3,Type12_3,Type2_3,Type3_3,Type4_3,Type5_3,Type6_3,Type7_3,Type8_3,Type9_3
0,2023-01-01 06:00:00,B_0,Cell0,0.189872,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01 11:00:00,B_0,Cell0,0.522096,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-01 12:00:00,B_0,Cell0,0.439968,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-01 13:00:00,B_0,Cell0,0.482723,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-01 23:00:00,B_0,Cell0,0.782511,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,4,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Add number of simultaneously working cells to prediction dataset as well
cwc = len([c for c in df_pred_bs.columns if "_0" in c])
df_pred_bs["Num Working Cell"] = 4- (df_pred_bs.isna().sum(1)//cwc)

df_pred_bs["Hour"] = df_pred_bs["Time"].dt.hour
df_pred_bs["Day"] = df_pred_bs["Time"].dt.day_of_week
df_pred_bs["Hour Sin"] = (np.sin(df_pred_bs["Hour"]*2*np.pi/24))
df_pred_bs["Hour Cos"] = (np.cos(df_pred_bs["Hour"]*2*np.pi/24))
df_pred_bs["Day Sin"] = (np.sin(df_pred_bs["Day"]*2*np.pi/7))
df_pred_bs["Day Cos"] = (np.cos(df_pred_bs["Day"]*2*np.pi/7))
df_pred_bs.drop(columns=["Hour","Day"],inplace=True)

df_pred_bs["Antennas"] = df_pred_bs["Antennas_0"]
df_pred_bs = df_pred_bs.drop(columns=["Antennas_0","Antennas_1", "Antennas_2", "Antennas_3"])
df_pred_bs = df_pred_bs.drop(columns=["CellName_0","CellName_1", "CellName_2", "CellName_3"])

In [12]:
df_pred_bs["Avg"] = 0.
for BS in df_pred["BS"].unique():
    available_data = df_bs[df_bs["BS"]==BS].sort_values("Time")
    filt = df_pred_bs[df_pred_bs["BS"]==BS].sort_values("Time")
    for idx,row in filt.iterrows():
        ret = available_data[available_data["Time"] < row["Time"]]["Avg"]
        if len(ret) > 0:
            df_pred_bs.loc[idx,"Avg"] = ret.iloc[-1]

In [13]:
df_pred_bs.head()

,Time,BS,load_0,ESMode1_0,ESMode2_0,ESMode3_0,ESMode4_0,ESMode5_0,ESMode6_0,Mode_0,Frequency_0,Bandwidth_0,TXpower_0,Type10_0,Type11_0,Type12_0,Type2_0,Type3_0,Type4_0,Type5_0,Type6_0,Type7_0,Type8_0,Type9_0,load_1,ESMode1_1,ESMode2_1,ESMode3_1,ESMode4_1,ESMode5_1,ESMode6_1,Mode_1,Frequency_1,Bandwidth_1,TXpower_1,Type10_1,Type11_1,Type12_1,Type2_1,Type3_1,Type4_1,Type5_1,Type6_1,Type7_1,Type8_1,Type9_1,load_2,ESMode1_2,ESMode2_2,ESMode3_2,ESMode4_2,ESMode5_2,ESMode6_2,Mode_2,Frequency_2,Bandwidth_2,TXpower_2,Type10_2,Type11_2,Type12_2,Type2_2,Type3_2,Type4_2,Type5_2,Type6_2,Type7_2,Type8_2,Type9_2,load_3,ESMode1_3,ESMode2_3,ESMode3_3,ESMode4_3,ESMode5_3,ESMode6_3,Mode_3,Frequency_3,Bandwidth_3,TXpower_3,Type10_3,Type11_3,Type12_3,Type2_3,Type3_3,Type4_3,Type5_3,Type6_3,Type7_3,Type8_3,Type9_3,Num Working Cell,Hour Sin,Hour Cos,Day Sin,Day Cos,Antennas,Avg
0,2023-01-01 06:00:00,B_0,0.189872,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.000000e+00,6.123234e-17,-0.781831,0.62349,4,58.258595
1,2023-01-01 11:00:00,B_0,0.522096,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.588190e-01,-9.659258e-01,-0.781831,0.62349,4,67.656951
2,2023-01-01 12:00:00,B_0,0.439968,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.224647e-16,-1.000000e+00,-0.781831,0.62349,4,67.656951
3,2023-01-01 13:00:00,B_0,0.482723,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-2.588190e-01,-9.659258e-01,-0.781831,0.62349,4,67.656951
4,2023-01-01 23:00:00,B_0,0.782511,0.0,0.0,0.0,0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-2.588190e-01,9.659258e-01,-0.781831,0.62349,4,72.918315


In [14]:
df_bs["is pred"]=False
df_pred_bs["is pred"]=True
df_all = pd.concat((df_bs,df_pred_bs),axis=0)
df_all

,Time,BS,load_0,ESMode1_0,ESMode2_0,ESMode3_0,ESMode4_0,ESMode5_0,ESMode6_0,Mode_0,Frequency_0,Bandwidth_0,TXpower_0,Type10_0,Type11_0,Type12_0,Type2_0,Type3_0,Type4_0,Type5_0,Type6_0,Type7_0,Type8_0,Type9_0,load_1,ESMode1_1,ESMode2_1,ESMode3_1,ESMode4_1,ESMode5_1,ESMode6_1,Mode_1,Frequency_1,Bandwidth_1,TXpower_1,Type10_1,Type11_1,Type12_1,Type2_1,Type3_1,Type4_1,Type5_1,Type6_1,Type7_1,Type8_1,Type9_1,load_2,ESMode1_2,ESMode2_2,ESMode3_2,ESMode4_2,ESMode5_2,ESMode6_2,Mode_2,Frequency_2,Bandwidth_2,TXpower_2,Type10_2,Type11_2,Type12_2,Type2_2,Type3_2,Type4_2,Type5_2,Type6_2,Type7_2,Type8_2,Type9_2,load_3,ESMode1_3,ESMode2_3,ESMode3_3,ESMode4_3,ESMode5_3,ESMode6_3,Mode_3,Frequency_3,Bandwidth_3,TXpower_3,Type10_3,Type11_3,Type12_3,Type2_3,Type3_3,Type4_3,Type5_3,Type6_3,Type7_3,Type8_3,Type9_3,Num Working Cell,Hour Sin,Hour Cos,Day Sin,Day Cos,Antennas,Energy,Avg,is pred
0,2023-01-01 01:00:00,B_0,0.487936,0.0,0.0,0.0,0,0.0,0.000000,1,365.000,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.258819,0.965926,-0.781831,0.62349,4,64.275037,0.000000,False
1,2023-01-01 02:00:00,B_0,0.344468,0.0,0.0,0.0,0,0.0,0.000000,1,365.000,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.500000,0.866025,-0.781831,0.62349,4,55.904335,64.275037,False
2,2023-01-01 03:00:00,B_0,0.193766,0.0,0.0,0.0,0,0.0,0.000000,1,365.000,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.707107,0.707107,-0.781831,0.62349,4,57.698057,60.089686,False
3,2023-01-01 04:00:00,B_0,0.222383,0.0,0.0,0.0,0,0.0,0.000000,1,365.000,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.866025,0.500000,-0.781831,0.62349,4,55.156951,59.292476,False
4,2023-01-01 05:00:00,B_0,0.175436,0.0,0.0,0.0,0,0.0,0.000000,1,365.000,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.965926,0.258819,-0.781831,0.62349,4,56.053812,58.258595,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26134,2023-01-02 19:00:00,B_1019,0.019940,0.0,0.0,0.0,0,0.0,0.627736,0,697.002,10,7.086398,0,1,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.965926,0.258819,0.000000,1.00000,2,NaN,0.000000,True
26135,2023-01-02 20:00:00,B_1019,0.019940,0.0,0.0,0.0,0,0.0,0.627909,0,697.002,10,7.086398,0,1,0,0,0,0

In [15]:
bs_names = list(df_all["BS"].unique())
ltr = LagFeatures(variables=["load_0", "load_1", "load_2", "load_3",
                             "ESMode1_0",	"ESMode2_0",	"ESMode6_0",
                             "ESMode1_1",	"ESMode2_1",	"ESMode6_1",
                             "ESMode1_2",	"ESMode2_2",	"ESMode6_2",
                             "ESMode1_3",	"ESMode2_3",	"ESMode6_3",
                             ],
                  freq=["1h", "2h","3h"])
ltr2 = LagFeatures(variables=["Energy"], freq=["1h","2h","1D"])
df_all = df_all.fillna(-999)
df_all_ = None

for i,bs_name in enumerate(bs_names):
    df_b = df_all[df_all["BS"]==bs_name]
    df_b["Time"] = pd.to_datetime(df_b["Time"])
    df_b = df_b.set_index("Time")
    
    df_b = ltr2.fit_transform(df_b)
    df_b = ltr.fit_transform(df_b)

    df_b = df_b.reset_index()
    df_b = df_b.replace(-999,np.nan)
    
    print(f"{(i+1)/len(bs_names)*100:.2f}%")

    if df_all_ is None:
        df_all_ = df_b.copy()
    else:
        df_all_ = pd.concat((df_all_,df_b),axis=0)

0.10%
0.20%
0.29%
0.39%
0.49%
0.59%
0.69%
0.78%
0.88%
0.98%
1.08%
1.18%
1.27%
1.37%
1.47%
1.57%
1.67%
1.76%
1.86%
1.96%
2.06%
2.16%
2.25%
2.35%
2.45%
2.55%
2.65%
2.75%
2.84%
2.94%
3.04%
3.14%
3.24%
3.33%
3.43%
3.53%
3.63%
3.73%
3.82%
3.92%
4.02%
4.12%
4.22%
4.31%
4.41%
4.51%
4.61%
4.71%
4.80%
4.90%
5.00%
5.10%
5.20%
5.29%
5.39%
5.49%
5.59%
5.69%
5.78%
5.88%
5.98%
6.08%
6.18%
6.27%
6.37%
6.47%
6.57%
6.67%
6.76%
6.86%
6.96%
7.06%
7.16%
7.25%
7.35%
7.45%
7.55%
7.65%
7.75%
7.84%
7.94%
8.04%
8.14%
8.24%
8.33%
8.43%
8.53%
8.63%
8.73%
8.82%
8.92%
9.02%
9.12%
9.22%
9.31%
9.41%
9.51%
9.61%
9.71%
9.80%
9.90%
10.00%
10.10%
10.20%
10.29%
10.39%
10.49%
10.59%
10.69%
10.78%
10.88%
10.98%
11.08%
11.18%
11.27%
11.37%
11.47%
11.57%
11.67%
11.76%
11.86%
11.96%
12.06%
12.16%
12.25%
12.35%
12.45%
12.55%
12.65%
12.75%
12.84%
12.94%
13.04%
13.14%
13.24%
13.33%
13.43%
13.53%
13.63%
13.73%
13.82%
13.92%
14.02%
14.12%
14.22%
14.31%
14.41%
14.51%
14.61%
14.71%
14.80%
14.90%
15.00%
15.10%
15.20%
15.29%
15.39%
15

In [16]:
cols_ordered = []
all_cols = list(df_all_.columns)
for col_name in all_cols:
    filtered = list(filter(lambda cn: col_name in cn,all_cols))
    cols_ordered += filtered
    all_cols = [x for x in all_cols if x not in filtered]

df_all_ = df_all_[cols_ordered]

In [17]:
df_all_.head()

,Time,BS,load_0,load_0_lag_1h,load_0_lag_2h,load_0_lag_3h,ESMode1_0,ESMode1_0_lag_1h,ESMode1_0_lag_2h,ESMode1_0_lag_3h,ESMode2_0,ESMode2_0_lag_1h,ESMode2_0_lag_2h,ESMode2_0_lag_3h,ESMode3_0,ESMode4_0,ESMode5_0,ESMode6_0,ESMode6_0_lag_1h,ESMode6_0_lag_2h,ESMode6_0_lag_3h,Mode_0,Frequency_0,Bandwidth_0,TXpower_0,Type10_0,Type11_0,Type12_0,Type2_0,Type3_0,Type4_0,Type5_0,Type6_0,Type7_0,Type8_0,Type9_0,load_1,load_1_lag_1h,load_1_lag_2h,load_1_lag_3h,ESMode1_1,ESMode1_1_lag_1h,ESMode1_1_lag_2h,ESMode1_1_lag_3h,ESMode2_1,ESMode2_1_lag_1h,ESMode2_1_lag_2h,ESMode2_1_lag_3h,ESMode3_1,ESMode4_1,ESMode5_1,ESMode6_1,ESMode6_1_lag_1h,ESMode6_1_lag_2h,ESMode6_1_lag_3h,Mode_1,Frequency_1,Bandwidth_1,TXpower_1,Type10_1,Type11_1,Type12_1,Type2_1,Type3_1,Type4_1,Type5_1,Type6_1,Type7_1,Type8_1,Type9_1,load_2,load_2_lag_1h,load_2_lag_2h,load_2_lag_3h,ESMode1_2,ESMode1_2_lag_1h,ESMode1_2_lag_2h,ESMode1_2_lag_3h,ESMode2_2,ESMode2_2_lag_1h,ESMode2_2_lag_2h,ESMode2_2_lag_3h,ESMode3_2,ESMode4_2,ESMode5_2,ESMode6_2,ESMode6_2_lag_1h,ESMode6_2_lag_2h,ESMode6_2_lag_3h,Mode_2,Frequency_2,Bandwidth_2,TXpower_2,Type10_2,Type11_2,Type12_2,Type2_2,Type3_2,Type4_2,Type5_2,Type6_2,Type7_2,Type8_2,Type9_2,load_3,load_3_lag_1h,load_3_lag_2h,load_3_lag_3h,ESMode1_3,ESMode1_3_lag_1h,ESMode1_3_lag_2h,ESMode1_3_lag_3h,ESMode2_3,ESMode2_3_lag_1h,ESMode2_3_lag_2h,ESMode2_3_lag_3h,ESMode3_3,ESMode4_3,ESMode5_3,ESMode6_3,ESMode6_3_lag_1h,ESMode6_3_lag_2h,ESMode6_3_lag_3h,Mode_3,Frequency_3,Bandwidth_3,TXpower_3,Type10_3,Type11_3,Type12_3,Type2_3,Type3_3,Type4_3,Type5_3,Type6_3,Type7_3,Type8_3,Type9_3,Num Working Cell,Hour Sin,Hour Cos,Day Sin,Day Cos,Antennas,Energy,Energy_lag_1h,Energy_lag_2h,Energy_lag_1D,Avg,is pred
0,2023-01-01 01:00:00,B_0,0.487936,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0,0.0,0.0,NaN,NaN,NaN,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.258819,0.965926,-0.781831,0.62349,4,64.275037,NaN,NaN,NaN,0.000000,False
1,2023-01-01 02:00:00,B_0,0.344468,0.487936,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0,0.0,0.0,0.0,NaN,NaN,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.500000,0.866025,-0.781831,0.62349,4,55.904335,64.275037,NaN,NaN,64.275037,False
2,2023-01-01 03:00:00,B_0,0.193766,0.344468,0.487936,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,0.0,NaN,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.707107,0.707107,-0.781831,0.62349,4,57.698057,55.904335,64.275037,NaN,60.089686,False
3,2023-01-01 04:00:00,B_0,0.222383,0.193766,0.344468,0.487936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1,365.0,20,6.875934,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [18]:
# Save training dataset
df_all_[df_all_["is pred"]==False].to_csv("data/processed/multi_cell_training_base.csv",index=False)

# Save prediction dataset
df_all_[df_all_["is pred"]==True].to_csv("data/processed/multi_cell_prediction_base.csv",index=False)